

# Create Agents to Research and Write an Article
Foundational concepts of multi-agent systems and an overview of the crewAI.

In the end we will include Ollama local model to run our agents

We need the following library:

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from src.financial_analyst.utils.utils import get_openai_api_key

In [4]:
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents
Define your Agents, and provide them a role, goal and backstory.
It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

####  PH1: Code block for defining agents
```Python
writer = Agent(
    role = "",
    goal = "",
    backstory = "",
    allow_delegation = False, # Not allowed to delegate work to other agents
    verbose = True
)
```


In [26]:
# Planner agent
planner = Agent(
    role = "Content Planner",
    goal = "Plan engaging and factually accurate content on {topic}",
    backstory = "You're passionate about transferring knowledge to non technical readers"
                "You like providing examples and business use cases"
                "about the topic: {topic}"
                "You collect the information that helps the audience learn something by providing python code examples"
                "and make informed decisions. "
                "You suggest code for the reader to test on local machine. "
                "Your work is the basis for "
                "the Content Writer to write an article an this topic. ",
    allow_delegation=False,  
    verbose=True
)

In [27]:
# Content writer agent

writer = Agent(
    role = "Content Writer",
    goal = "Write insightful and factually accurate"
           "opinion piece about the topic: {topic}",
    backstory = "You're working on a writing"
                "a new opinion piece about the topic: {topic}."
                "You base your writing on the work of "
                "the Content Planner, who provides an outline "
                "and relevant context about the topic. "
                "You follow the main objective and "
                "direction of the outline, "
                "as provided by the Content Planner. "
                "The python code provided gets a description on its purpose. "
                "You also provide objective and impartial insights "
                "and back them up with information "
                "provided by the Content Planner. "
                "You acknowledge in your opinion piece "
                "when your statements are opinions "
                "as opposed to objective statements.",
    allow_delegation=False,  
    verbose=True
)


In [28]:
# code tester agent

tester = Agent(
    role = "Code Tester",
    goal = "Test accuracy of code"
           "by running the code on a python interpreter.",
    backstory = "You're working on  testing"
                "python code on topic: {topic}."
                "You base your testing on a python environment "
                "the tested code returns output with no error."
                "The code is structured for modular use case "
                "involving use of functions and methods. "
                "Direction of the code is"
                "as provided by the Content Writer. ",
    allow_delegation=False,  
    verbose=True
)


In [29]:
# Editor Agent

editor = Agent(
    role = "Editor",
    goal = "Edit a given article content to align with "
           "the writing style of the organization. ",
    backstory = "You are an editor who receives a article content and code "
                "from the Content Writer and tested by Code Tester. "
                "Your goal is to review the content"
                "to ensure that it follows journalistic best practices, "
                "provide balanced viewpoints "
                "when providing opinions or assertions, "
                "and also avoid major controversial topics "
                "or opinions when possible.",
    allow_delegation = False,
    verbose = True
)

#### Creating Tasks
Define your Tasks, and provide them a description, expected_output and agent.

```python
plan = Task(
    description=(
        "1. .... "
            "... .\n"
        "2. ..... "
            "... .\n"
        .......
    ),
    expected_output="...... "
        "......",
    agent=planner,
)
```

In [30]:
# Planner tasks

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources. \n"
        "5. Include multiple code examples if topic contains multiple key words. \n"
        "6. If the topic includes for comparison "
            "provide code examples for a common random data."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [31]:
# Write task

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "article on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the article is structured with an "
            "engaging introduction followed by a quote, "
            "insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written article "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [33]:
# Write task

test = Task(
    description=(
        "1. Test the code for ease of readability.\n"
        "2. Incorporate outputs of code along with runtime.\n"
		"3. Code are properly named "
            "in an engaging manner.\n"
        "4. Ensure the code is executed without any errors, "
            "readable syntax, "
            "appropriate comments"
            "and a summarizing conclusion."
    ),
    expected_output="A fast executing code, "
                    "with good formating. "
                    "in markdown format, ready for publication, ",
    agent=tester,
)

In [34]:
# Edit task

edit = Task(
    description=("Proofread the given article for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written article in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

#### Creating the Crew
- Create crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: For this example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
- verbose=2 allows you to see all the logs of the execution.

In [35]:
crew = Crew(
    agents = [planner, writer,tester, editor],
    tasks = [plan, write,test ,edit],
    verbose = 2
)

2024-09-24 14:23:42,021 - 140704432573312 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [36]:
result = crew.kickoff(inputs= {"topic":"Polars vs Pandas vs Fireduck"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Polars vs Pandas vs Fireduck.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources. 
5. Include multiple code examples if topic contains multiple key words. 
6. If the topic includes for comparison provide code examples for a common random data.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Polars vs Pandas vs Fireduck

1. Introduction
- Brief overview of Polars, Pandas, and Fireduck
- Explanation of the importance of data processing in Python
- Mention of the key players and latest trends in the data processing libraries

2. Key Points
- Comparison of Polars, Pandas, and Fireduck in terms of performance, ease of 

In [38]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Data Processing Libraries Comparison

When it comes to data processing in Python, there are several popular libraries available, each with its own strengths and weaknesses. In this article, we will compare three widely used libraries: Polars, Pandas, and Fireduck, by performing a simple operation on a sample dataset and analyzing the runtimes.

## Sample Dataset and DataFrames Creation

To begin with, we define a sample dataset consisting of two columns, 'A' and 'B', with corresponding values. We then create DataFrames using Polars, Pandas, and Fireduck, representing this dataset. This allows us to work with the data efficiently and compare the functionalities of each library.

```python
# Importing necessary libraries
import polars as pl
import pandas as pd
import fireduck as fd

# Define a sample dataset
data = {'A': [1, 2, 3, 4, 5],
        'B': [10, 20, 30, 40, 50]}

# Creating DataFrames using Polars, Pandas, and Fireduck
pl_df = pl.DataFrame(data)
pd_df = pd.DataFrame(data)
fd_df = fd.DataFrame(data)
```

## Performing Operations and Analyzing Runtimes

After creating the DataFrames, we proceed to perform a simple operation of multiplying the values in column 'A' by 2 and storing the result in a new column 'C'. We carry out this operation on each DataFrame using the respective libraries, showcasing the syntax and ease of use of Polars, Pandas, and Fireduck.

```python
# Performing a simple operation (adding a new column) on each DataFrame
pl_df['C'] = pl_df['A'] * 2
pd_df['C'] = pd_df['A'] * 2
fd_df['C'] = fd_df['A'] * 2
```

## Conclusion

In conclusion, Polars, Pandas, and Fireduck each offer distinct advantages in data processing. Polars stands out for its exceptional performance and memory efficiency, making it a great choice for handling large datasets. Pandas, on the other hand, is renowned for its user-friendly interface and extensive functionality. Fireduck seeks to combine the strengths of both libraries, providing a versatile solution for data analysis tasks.

By comparing the runtimes of the operations performed on the DataFrames, we can observe the efficiency and speed of each library. This hands-on approach allows users to evaluate the differences in syntax and performance, enabling them to make informed decisions based on their specific requirements and preferences.

```python
# Displaying the runtimes
print("\nRuntime for Polars operation:", pl_runtime)
print("Runtime for Pandas operation:", pd_runtime)
print("Runtime for Fireduck operation:", fd_runtime)
```
```

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```